In [1]:
import os
import pandas as pd
import warnings

from sqlalchemy import create_engine
from sqlalchemy.sql import text
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings('ignore')

### Загрузка Данных

In [2]:
banks_df = pd.read_excel('../data/banks.xlsx')
offers_df = pd.read_excel('../data/offers.xlsx')

In [3]:
banks_df.head()

,id,name,rate,online_rate,term,amount_from,amount_to,offer_count,final_rate
0,1,Транскапиталбанк,24.5,0.00,365 дней,100000,3000000.0,3,24.50
1,2,Локо-Банк,24.5,0.00,182 дня,300000,5000000.0,1,24.50
2,3,Газпромбанк,24.0,0.01,от 1 дня,1,1500000.0,0,24.00
3,4,Альфа-Банк,24.0,24.00,от 1 дня,1,1500000.0,0,24.00
4,5,Т-Банк,23.0,24.13,2 – 24 месяцев,50000,NaN,1,24.13


In [4]:
offers_df.head()

,bank_id,bank_name,rate,online_rate,term,amount_from,amount_to,final_rate
0,1,Транскапиталбанк,24.40,0.00,180 дней,100000.0,3000000.0,24.40
1,1,Транскапиталбанк,22.55,0.00,90 дней,100000.0,3000000.0,22.55
2,1,Транскапиталбанк,10.25,0.00,1110 дней,100000.0,3000000.0,10.25
3,2,Локо-Банк,22.50,0.00,370 дней,300000.0,5000000.0,22.50
4,5,Т-Банк,20.00,20.17,2 – 24 месяцев,50000.0,NaN,20.17


### Подключение к Postgres

In [5]:
# определяем параметры подлчения к postgres
user_name = os.getenv("POSTGRES_DB_USER")
pwd = os.getenv("POSTGRES_DB_PASSWORD")
port = os.getenv("POSTGRES_DB_HOST_PORT")
db_name = os.getenv("POSTGRES_DB_NAME")
host = 'localhost'

# названия таблиц
banks_table_name = 'banks'
offers_table_name = 'offers'

# создаем подключение к postgres
engine = create_engine(f'postgresql://{user_name}:{pwd}@{host}:{port}/{db_name}')

### Создаем Таблицы в БД
Для создание таблицы можно сипользовать несколько способов
- Напрямую вставлять данные
- Используя `pd.io.sql.get_schema`

In [6]:
# способ 1
banks_df.to_sql(name=banks_table_name, con=engine, if_exists='replace')

186

In [7]:
offers_df.to_sql(name=offers_table_name, con=engine, if_exists='replace')

795

In [42]:
# способ 2
print(pd.io.sql.get_schema(banks_df, name=banks_table_name))

CREATE TABLE "banks" (
"id" INTEGER,
  "name" TEXT,
  "rate" REAL,
  "online_rate" REAL,
  "term" TEXT,
  "amount_from" INTEGER,
  "amount_to" REAL,
  "offer_count" INTEGER,
  "final_rate" REAL
)


In [18]:
# создаем таблицы в БД, если таблицы уже созданые, выведет ошибку, поэтому используем try-except
try:
    with engine.connect() as connection:
        connection.execute(
            # используем text из sqlalchemy, так как он преоразует строку создания таблицы в нужный формат
            # без этого, ругается!
            text(pd.io.sql.get_schema(offers_df, name=offers_table_name, con=engine))
        )
        connection.execute(
            text(pd.io.sql.get_schema(banks_df, name=banks_table_name, con=engine))
        )
except:
    print('Одна из таблиц уже существует! Удали ее!')


Одна из таблиц уже существует! Удали ее!


### Получение Данные из БД

In [8]:
query = """
SELECT
    name,
    rate,
    online_rate,
    term,
    amount_from,
    amount_to,
    offer_count,
    final_rate
FROM
    banks
ORDER BY
    rate DESC
LIMIT 10
"""

# формат исполнения запроса
with engine.connect() as connection:
    result = connection.execute(text(query))
    rows = result.fetchall()
    columns = result.keys()

result_dict = [dict(zip(columns, row)) for row in rows]
result_dict

[{'name': 'ПСБ',
  'rate': 30.0,
  'online_rate': 0.0,
  'term': '91 – 367 дней',
  'amount_from': 50000,
  'amount_to': 50000.0,
  'offer_count': 9,
  'final_rate': 30.0},
 {'name': 'Гарант-инвест',
  'rate': 26.5,
  'online_rate': 0.0,
  'term': '181 – 370 дней',
  'amount_from': 1000000,
  'amount_to': None,
  'offer_count': 12,
  'final_rate': 26.5},
 {'name': 'ОЗОН Банк',
  'rate': 25.0,
  'online_rate': 0.0,
  'term': 'от 1 дня',
  'amount_from': 1,
  'amount_to': None,
  'offer_count': 1,
  'final_rate': 25.0},
 {'name': 'Инвестторгбанк',
  'rate': 25.0,
  'online_rate': 0.0,
  'term': '91 – 370 дней',
  'amount_from': 50000,
  'amount_to': None,
  'offer_count': 6,
  'final_rate': 25.0},
 {'name': 'БыстроБанк',
  'rate': 25.0,
  'online_rate': 0.0,
  'term': '1830 – 1840 дней',
  'amount_from': 10000,
  'amount_to': 5000000.0,
  'offer_count': 5,
  'final_rate': 25.0},
 {'name': 'Банк Уралсиб',
  'rate': 24.65,
  'online_rate': 0.0,
  'term': '91 – 732 дней',
  'amount_from': 5

In [9]:
GRAPH_BUTTON_TEXT = "Запросить График 📈"

# ключевая ставка
INTEREST_RATE_BUTTON_TEXT = "Запросить процентную ставку 📊"
INTEREST_RATE_RESPONSE_HEADER = "Вот топ-10 Предложений на Сегодня 💼💰\n"
INTEREST_RATE_RESPONSE_FOOTER = "\n💡Спасибо, что ознакомились с предложениями! Надеемся, эта информация поможет вам принять лучшее решение 🧠💡"

INTEREST_RATE_RESPONSE_STRUCTURE = """\n
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: {name}
Ставка: {rate}%
Онлайн Ставка: {online_rate}%
Срок: {term}
Минимальная Сумма: {amount_from} RUB
Максимальная Сумма: {amount_to} RUB
Количество Предложений: {offer_count}
Итоговая Ставка: {final_rate}%
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
"""

In [10]:
response = INTEREST_RATE_RESPONSE_HEADER
for bank_data in result_dict:
    response += INTEREST_RATE_RESPONSE_STRUCTURE.format(**bank_data)

response += INTEREST_RATE_RESPONSE_FOOTER
print(response)

Вот топ-10 Предложений на Сегодня 💼💰


💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: ПСБ
Ставка: 30.0%
Онлайн Ставка: 0.0%
Срок: 91 – 367 дней
Минимальная Сумма: 50000 RUB
Максимальная Сумма: 50000.0 RUB
Количество Предложений: 9
Итоговая Ставка: 30.0%
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸


💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: Гарант-инвест
Ставка: 26.5%
Онлайн Ставка: 0.0%
Срок: 181 – 370 дней
Минимальная Сумма: 1000000 RUB
Максимальная Сумма: None RUB
Количество Предложений: 12
Итоговая Ставка: 26.5%
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸


💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: ОЗОН Банк
Ставка: 25.0%
Онлайн Ставка: 0.0%
Срок: от 1 дня
Минимальная Сумма: 1 RUB
Максимальная Сумма: None RUB
Количество Предложений: 1
Итоговая Ставка: 25.0%
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸


💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: Инвестторгбанк
Ставка: 25.0%
Онлайн Ставка: 0.0%
Срок: 91 – 370 дней
Минимальная Сумма: 50000 RUB
Максимальная Сумма: None RUB
Количество Предложений: 6
Итоговая Ставка: 25.0%
💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸


💸💎💸💎💸💎💸💎💸💎💸💎💸💎💸
Банк: БыстроБанк
Ставка: 25.0%
Онлайн Ставка: 0.0%
Срок: 1830 – 1840 дней
Минимальная Сумма: 10000 R